In [ ]:
!pip install transformers datasets evaluate librosa torchaudio jiwer

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write

In [ ]:
from datasets import load_dataset, DatasetDict


dataset = load_dataset("FBK-MT/Speech-MASSIVE-test", 'pl-PL', split='test', trust_remote_code=True)
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'locale', 'partition', 'scenario', 'scenario_str', 'intent_idx', 'intent_str', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments', 'tokens', 'labels', 'audio', 'path', 'is_transcript_reported', 'is_validated', 'speaker_id', 'speaker_sex', 'speaker_age', 'speaker_ethnicity_simple', 'speaker_country_of_birth', 'speaker_country_of_residence', 'speaker_nationality', 'speaker_first_language'],
    num_rows: 2974
})


In [ ]:
dataset = dataset.select_columns(['utt','audio'])
dataset[0]

{'id': '11861',
 'utt': 'jaki film jest teraz najwyżej oceniany',
 'audio': {'path': 'dab7ab8b100ece47b0a1e822a6beac5b.wav',
  'array': array([-6.98491931e-10,  8.14907253e-10, -8.14907253e-10, ...,
         -7.34902860e-04, -6.60274061e-04, -4.50702908e-04]),
  'sampling_rate': 16000}}

In [ ]:
import librosa
import os
from transformers import WhisperProcessor


processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")

def preprocess_function(batch):

    audio = batch["audio"]
    input_features = processor(
        audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt"
    ).input_features

    batch["input_features"] = input_features
    batch["input_ids"] = processor.tokenizer(batch["utt"]).input_ids

    return batch

In [ ]:
processed_dataset = dataset.map(preprocess_function)
# processed_dataset = processed_dataset.remove_columns(["utt", "audio"])

train_val_dataset = processed_dataset.train_test_split(test_size=0.2, seed=42)

train_val_split = train_val_dataset["train"].train_test_split(test_size=0.25, seed=42)

processed_dataset = DatasetDict({
    "train": train_val_split["train"],
    "validation": train_val_split["test"],
    "test": train_val_dataset["test"]
})

In [ ]:
import torch
from transformers import WhisperForConditionalGeneration


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.to(device)


Using device: cuda


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 384)
      (layers): ModuleList(
        (0-3): 4 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=384, out_features=384, bias=False)
            (v_proj): Linear(in_features=384, out_features=384, bias=True)
            (q_proj): Linear(in_features=384, out_features=384, bias=True)
            (out_proj): Linear(in_features=384, out_features=384, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          

In [ ]:
def transcribe(batch):
    input_features = batch["input_features"]
    input_features = torch.tensor(input_features).to(device)

    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription


results = processed_dataset["test"].select(range(5)).map(lambda x: {"transcription": transcribe(x)})

for result in results:
    print(f"Original Text: {result['utt']}")
    print(f"Transcription: {result['transcription'].lower()}")
    print("-" * 50)


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Original Text: wyślij maila do mojego brata i przypomnij o rocznicy ślubu
Transcription: wyślij maila do mojego bryata i przypomnij o lec nicy ślubu
--------------------------------------------------
Original Text: przypomnij mi o jutrzejszym spotkaniu godzinę wcześniej
Transcription: przypomnij mi o jutrzejszym spotkaniu godziny wcześniej
--------------------------------------------------
Original Text: graj plejlistę boba dylana
Transcription: graj playlistę boba delana
--------------------------------------------------
Original Text: graj ale jazz autorki sanah
Transcription: graj ale jazz autorki sanah
--------------------------------------------------
Original Text: olly posłuchajmy sto jeden i trzy f. m.
Transcription: olly posłuchajmy z to jeden i trzy f. m.
--------------------------------------------------


In [ ]:
from evaluate import load


wer_metric = load("wer")

results = processed_dataset["test"].map(lambda x: {"transcription": transcribe(x)})

wer = wer_metric.compute(
    predictions=results["transcription"], references=results["utt"]
)


print(f"Word Error Rate (WER) on the test set: {wer:.4f}")

Map:   0%|          | 0/595 [00:00<?, ? examples/s]

Word Error Rate (WER) on the test set: 0.8435


Word Error Rate (WER) on the test set: 0.8435

In [ ]:
from transformers import TrainingArguments
from transformers import DataCollatorWithPadding

training_args = TrainingArguments(
    output_dir="./whisper-tiny-polish",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
)

def data_collator(features):
    input_features = [{"input_features": torch.tensor(feature["input_features"]).squeeze(0)} for feature in features]
    input_features = processor.feature_extractor.pad(input_features, return_tensors="pt")

    labels = [{"input_ids": feature["input_ids"]} for feature in features]
    labels = processor.tokenizer.pad(labels, return_tensors="pt")

    return {
        "input_features": input_features["input_features"],
        "labels": labels["input_ids"],
    }

In [ ]:
from transformers import Trainer
import os

os.environ["WANDB_DISABLED"] = "true"

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    data_collator=data_collator,
    tokenizer=processor.tokenizer,
)
trainer.train()

<ipython-input-40-2f466d033502>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,0.324271
2,No log,0.328852
3,No log,0.325428
4,No log,0.316187
5,0.008300,0.318386


TrainOutput(global_step=560, training_loss=0.007558951353920358, metrics={'train_runtime': 2279.4663, 'train_samples_per_second': 3.913, 'train_steps_per_second': 0.246, 'total_flos': 2.196004552704e+17, 'train_loss': 0.007558951353920358, 'epoch': 5.0})

In [ ]:
results = processed_dataset["test"].map(lambda x: {"transcription": transcribe(x)})

wer = wer_metric.compute(
    predictions=results["transcription"], references=results["utt"]
)


print(f"Word Error Rate (WER) on the test set: {wer:.4f}")

Map:   0%|          | 0/595 [00:00<?, ? examples/s]

Word Error Rate (WER) on the test set: 0.3216


In [ ]:
model.push_to_hub("whisper-tiny-polish")
processor.push_to_hub("whisper-tiny-polish")

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/gs224/whisper-tiny-polish/commit/eb54d908839e48e9d829d21012e21b509e00618f', commit_message='Upload processor', commit_description='', oid='eb54d908839e48e9d829d21012e21b509e00618f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/gs224/whisper-tiny-polish', endpoint='https://huggingface.co', repo_type='model', repo_id='gs224/whisper-tiny-polish'), pr_revision=None, pr_num=None)